# Install libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# clone hifuku
parent_dir = '/content/drive/MyDrive' # install location
%cd $parent_dir
!git clone https://github.com/onnonuro/mmmetry.git

/content/drive/MyDrive
fatal: destination path 'mmmetry' already exists and is not an empty directory.


In [3]:
# install required dependencies.
root = f'{parent_dir}/mmmetry'
%cd $root
!pip install -e .

/content/drive/MyDrive/mmmetry
Obtaining file:///content/drive/MyDrive/mmmetry
  Preparing metadata (setup.py) ... done
  Attempting uninstall: mmmetry
    Found existing installation: mmmetry 0.1.0
    Uninstalling mmmetry-0.1.0:
      Successfully uninstalled mmmetry-0.1.0
  Running setup.py develop for mmmetry


In [3]:
# Importing necessary libraries and modules for data handling, visualization, and mathematical operations.
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
from glob import glob
import time

import csv
import os.path
from pathlib import Path
import random
import math
import warnings
warnings.simplefilter('ignore')

import cv2
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 933120000
from PIL import Image, ImageDraw, ImageFont
# from xml.etree.ElementTree import parse

import joblib
from sklearn.manifold import TSNE
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from scipy.spatial import distance
from scipy.stats import entropy
from sklearn.neighbors import KernelDensity

from tqdm import tqdm

# import torch
# import ultralytics
from ultralytics import YOLO

import networkx as nx
from community import community_louvain
import mmmetry

# Set a sample image for analysis

In [ ]:
root = '.'

wsi_path = f'{root}/data/sample_neuropathy.jpeg'
scale = 0.2730  # Scale in µm/pixel for the WSI.

# Run Muscle Morphometry

In [ ]:
# Execute Hifuku
start_time = time.time()
result = mmmetry.main(root, wsi_path, scale=scale)
end_time = time.time()

# Calculate the total time taken
elapsed_time = int(end_time - start_time)
print(f"The analysis took {elapsed_time} seconds.")

NameError: name 'time' is not defined

In [7]:
# Uncomment to unassign resources in Google Colab when finished.
# from google.colab import runtime
# runtime.unassign()